In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm


class Esco:
    def __init__(self, language="en"):
        #Desired language, defaul is english
        self.language = language
        
    def get_esco_skills(self,job_url):
        """ Given the ESCO job_url return the skills for this job"""
        #define API URL
        job_url = "https://ec.europa.eu/esco/api/resource/occupation?uri=" + job_url + "&language=" + self.language
        response = requests.get(job_url)
        job = dict(json.loads(response.text))
        skills_links = []
        skills_responses = []
        skills = []
        #Get skills links
        for i in job["_links"]["hasEssentialSkill"]:
            skills_links.append(i["href"])
        
        #request for each skill
        for i in skills_links:
            skill_response = requests.get(i)
            skills_responses.append(skill_response.text)
        
        #get the skills
        for i in skills_responses:
            skills.append(dict(json.loads(i))["preferredLabel"]["en-us"])
        
        return skills
    def get_esco_job_title(self,job_url):
        #define API URL
        job_url = "https://ec.europa.eu/esco/api/resource/occupation?uri=" + job_url + "&language=" + self.language
        response = requests.get(job_url)
        job = dict(json.loads(response.text))
        title = job["title"]
        return title

In [2]:
esco = Esco(language="en")

In [3]:
occupation_df = pd.read_csv("../data/Esco Inglês/occupations_en.csv")

FileNotFoundError: [Errno 2] File b'../data/Esco Ingle\xcc\x82s/occupations_en.csv' does not exist: b'../data/Esco Ingle\xcc\x82s/occupations_en.csv'

In [ ]:
title = []
skills = []
amount = 300
start = 0
end = 10
while start < 3000:
    try:
        if start >= 2600:
            for i in tqdm(occupation_df["conceptUri"][start:]):
                title.append(esco.get_esco_job_title(i))
                skills.append(esco.get_esco_skills(i))
                start = 3000
        else:
            for i in tqdm(occupation_df["conceptUri"][start:end]):
                title.append(esco.get_esco_job_title(i))
                skills.append(esco.get_esco_skills(i))
                start += 10
                end += 10
    except:
        print("On interval:")
        print("Start:",start)
        print("End:", end)
        print("")
        print("Stopped on:",i)

In [ ]:
skills_df = pd.DataFrame()
skills_df["title"] = title
skills_df["skills"] = skills

In [ ]:
skills_df.head()